ДАТАСЕТ

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv("train.csv", index_col=0)
df.head()

,review,label
0,I think they really let the quality of the DVD...,0
1,I'm sorry but this is just awful. I have told ...,0
2,"The Japenese sense of pacing, editing and musi...",0
3,"In the '60's/'70's, David Jason was renowned f...",1
4,"""Hail The Woman"" is one of the most moving fil...",1


ПРИВЕДЕНИЕ К НИЖНЕМУ РЕГИСТРУ И ЛЕММАТИЗАЦИЯ

In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

en_stop = list(stopwords.words('english'))
lemmatizer = WordNetLemmatizer() 

def tokenize(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [t for t in tokens if
              re.match(r'[^\W\d]*$', t) and (len(t) > 2) and (t not in en_stop)]
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return tokens

tokens = df['review'].apply(tokenize)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/matveykormuskin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/matveykormuskin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/matveykormuskin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


WORD2VEC

In [98]:
from gensim.models import Word2Vec
model = Word2Vec(tokens, size=300, window=5, min_count=2, iter=5, sample=1e-3, sg=1)

ПРОВЕРКА БЛИЗОСТИ

In [94]:
print(model.wv.similarity('movie', 'film'),model.wv.similarity('movie', 'cat'))
print(model.wv.most_similar('film')[:10])

0.90458345 0.37244636
[('movie', 0.9045833945274353), ('splatterfest', 0.7891319990158081), ('towelhead', 0.7877262830734253), ('morbidly', 0.7875232696533203), ('atleast', 0.7847403883934021), ('categorize', 0.7838698625564575), ('filmaking', 0.7814050912857056), ('underestimated', 0.7813634872436523), ('belive', 0.7812625765800476), ('disclosure', 0.7811387181282043)]


In [ ]:
ВЕКТОРНОЕ ПРЕДСТАВЛЕНИЕ И РАЗДЕЛЕНИЕ ДАТАСЕТА

In [100]:
def encode(list_of_tokens):
    x = np.array([model.wv[t] for t in list_of_tokens if t in model.wv.vocab])
    return np.mean((x/np.std(x,axis=0)), axis=0)

fts = np.array([encode(t) for t in tokens])
print(fts.shape)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(fts, df.label.values,
                                                    test_size=0.2, shuffle=True)

(40000, 300)


КЛАССИФИКАЦИЯ

In [50]:
#clf=LogisticRegression(C=2.636650898730358, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False).fit(X_train,y_train)

In [101]:
clf=LogisticRegression(C=18,penalty='l2',solver='lbfgs',max_iter=1000).fit(X_train,y_train)

In [49]:
    from sklearn import linear_model, decomposition, datasets
    from sklearn.pipeline import Pipeline
    from sklearn.model_selection import GridSearchCV, cross_val_score
    from sklearn.preprocessing import StandardScaler

    # Create an scaler object
    sc = StandardScaler()

    # Create a pca object
    pca = decomposition.PCA()

    
    logistic = LogisticRegression(solver='lbfgs', max_iter=1000)
    # Create a pipeline of three steps. First, standardize the data.
    # Second, tranform the data with PCA.
    # Third, train a logistic regression on the data.
    pipe = Pipeline(steps=[('sc', sc),
                           ('pca', pca),
                           ('logistic', logistic)])

    # Create Parameter Space
    # Create a list of a sequence of integers from 1 to 30 (the number of features in X + 1)
    n_components = list(range(280,300,3))
    # Create a list of values of the regularization parameter
    C = np.logspace(0, 4, 20)
    # Create a list of options for the regularization penalty
    penalty = ['l2']
    # Create a dictionary of all the parameter options 
    # Note has you can access the parameters of steps of a pipeline by using '__’
    parameters = dict(pca__n_components=n_components,
                      logistic__C=C,
                      logistic__penalty=penalty)

    # Conduct Parameter Optmization With Pipeline
    # Create a grid search object
    clf = GridSearchCV(pipe, parameters)

    # Fit the grid search
    clf.fit(X_train, y_train)
    # View The Best Parameters
    print('Best Penalty:', clf.best_estimator_.get_params()['logistic__penalty'])
    print('Best C:', clf.best_estimator_.get_params()['logistic__C'])
    print('Best Number Of Components:', clf.best_estimator_.get_params()['pca__n_components'])
    print(); print(clf.best_estimator_.get_params()['logistic'])

    # Use Cross Validation To Evaluate Model
    #CV_Result = cross_val_score(clf, X_train, y_train, cv=4, n_jobs=-1)
    #print(); print(CV_Result)
    #print(); print(CV_Result.mean())
    #print(); print(CV_Result.std())

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best Penalty: l2
Best C: 2.636650898730358
Best Number Of Components: 298

LogisticRegression(C=2.636650898730358, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


РЕЗУЛЬТАТЫ

In [102]:
from sklearn.metrics import classification_report

predicts = clf.predict(X_train)
print('Train\n', classification_report(y_train, predicts))

predicts = clf.predict(X_test)
print('Test\n', classification_report(y_test, predicts))

Train
               precision    recall  f1-score   support

           0       0.89      0.88      0.88     16035
           1       0.88      0.89      0.88     15965

    accuracy                           0.88     32000
   macro avg       0.88      0.88      0.88     32000
weighted avg       0.88      0.88      0.88     32000

Test
               precision    recall  f1-score   support

           0       0.89      0.87      0.88      4032
           1       0.87      0.89      0.88      3968

    accuracy                           0.88      8000
   macro avg       0.88      0.88      0.88      8000
weighted avg       0.88      0.88      0.88      8000



ЗАГРУЗКА ОТВЕТА

In [92]:
test1 = pd.read_csv('test.csv', index_col=0)
tokens1 = test1['review'].apply(tokenize)
fts1 = np.array([encode(t) for t in tokens1])

predicted = clf.predict(fts1)

pd.DataFrame({'Predicted': predicted}).to_csv('solution.csv', index_label='Id')